<a href="https://colab.research.google.com/github/varshaelza/Text-Summarization-Categorization/blob/main/rbmtrial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
% cd drive/My\ Drive/BBC Dataset

/content/drive/My Drive/BBC Dataset


***LOADING DATASET***

In [3]:
import pandas as pd
import numpy as np
traindata = pd.read_csv('/content/drive/MyDrive/BBC Dataset/fuzzysampletrain.csv')
testdata = pd.read_csv('/content/drive/MyDrive/BBC Dataset/fuzzysampletest.csv')

In [ ]:
traindata

,Unnamed: 0,News,Summary,Category_id,Type
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport
...,...,...,...,...,...
1775,1775,India seeks to boost construction\n \n India h...,"He said, ""No country can develop on the basis ...",0,business
1776,1776,Child access laws shake-up\n \n Parents who re...,Curfews and community service orders were othe...,2,politics
1777,1777,Parliament's record of scandal\n \n In a locke...,But divorces are by no means the only document...,2,politics
1778,1778,Prutton poised for lengthy FA ban\n \n Southam...,"He was out of order but he knows that,"" said R...",3,sport


In [ ]:
testdata

,Unnamed: 0,News,Summary,Category_id,Type
0,1288,Labour battle plan 'hides Blair'\n \n The Tori...,Mr Milburn said the economy would take centre ...,2,politics
1,1170,Tutu's Guantanamo release call\n \n Archbishop...,The South African archbishop said detentions w...,2,politics
2,1388,Holmes starts 2005 with GB events\n \n Kelly H...,Holmes will make her first track appearance on...,3,sport
3,1109,Tory expert denies defeatism\n \n The Conserva...,Mr Howard appointed Mr Crosby as his elections...,2,politics
4,1851,Movie body targets children's PCs\n \n The bod...,The Movie Association for America's (MPAA) fre...,4,tech
...,...,...,...,...,...
440,1117,NHS waiting time target is cut\n \n Hospital w...,"""But let us not forget two thirds of those wai...",2,politics
441,1234,Baron Kinnock makes Lords debut\n \n Former La...,Former Labour leader Neil Kinnock has official...,2,politics
442,1720,Player burn-out worries Robinson\n \n England ...,England coach Andy Robinson says English rugby...,3,sport
443,1230,Blunkett hints at election call\n \n Ex-Home S...,He clarified that he meant he would be in his ...,2,politics


In [4]:
df=pd.concat([traindata, testdata])

In [5]:
df


,Unnamed: 0,News,Summary,Category_id,Type
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport
...,...,...,...,...,...
440,1117,NHS waiting time target is cut\n \n Hospital w...,"""But let us not forget two thirds of those wai...",2,politics
441,1234,Baron Kinnock makes Lords debut\n \n Former La...,Former Labour leader Neil Kinnock has official...,2,politics
442,1720,Player burn-out worries Robinson\n \n England ...,England coach Andy Robinson says English rugby...,3,sport
443,1230,Blunkett hints at election call\n \n Ex-Home S...,He clarified that he meant he would be in his ...,2,politics


***PREPROCESSING***

all imports and downloads

In [6]:
import nltk
import re
!sudo pip install nltk==3.4
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
from nltk.corpus import stopwords
from string import punctuation

nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

from nltk.tokenize import word_tokenize

from nltk.util import ngrams

from numpy import dot
from numpy.linalg import norm

! pip install rake-nltk
from rake_nltk import Rake

from nltk.tag import pos_tag
from collections import Counter

     |████████████████████████████████| 1.4MB 4.1MB/s 
  Created wheel for nltk: filename=nltk-3.4-cp37-none-any.whl size=1436383 sha256=ec3fe6c441381ed79566a6473a8097c4ce49b7ba193dacdb5d30f0980d3462c5
  Stored in directory: /root/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package universal_tagset to /root/nltk_data..

Stopwords removal

In [7]:
STOPWORDS = set(stopwords.words('english'))
def rem_stopwords(text):
    return " ".join([word.lower() for word in str(text).split(" ") if word not in STOPWORDS and word not in punctuation and not word.isdigit()])

df["rem_stopwords"] = df["News"].apply(lambda text: rem_stopwords(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...


Sentence tokenisation

In [8]:
df["sen_token"] = df["rem_stopwords"].apply(lambda text: sent_tokenize(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...


Porter Stemmer stemming

In [9]:
stemmer = PorterStemmer()

def stem_words(text):
    #return " ".join([stemmer.stem(word) for word in text.split()])
    nltk_stemmedList = []
    for word in text:
        nltk_stemmedList.append(stemmer.stem(word))
    return nltk_stemmedList
df["Porter_stemmed"] = df["sen_token"].apply(lambda text: stem_words(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...


Lemmatisation

In [10]:
wordnet_lemmatizer = WordNetLemmatizer()
def lemma(text):
  lemma_list=[]
  for word in text:
    lemma_list.append(wordnet_lemmatizer.lemmatize(word))
  return lemma_list
df["Lemmatize"] = df["Porter_stemmed"].apply(lambda text: lemma(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...


PoS tagging


In [ ]:
def postag(text):
  
  pos_list=[]
  for word in text:
    pos_list.append(nltk.pos_tag(word_tokenize(word),tagset='universal'))
     
  return pos_list
df["postag"] = df["Lemma"].apply(lambda text: postag(text))
df.head()

KeyError: ignored

***FEATURE EXTRACTION***

bigrams

In [11]:

stop_words = set(stopwords.words('english'))

def bi_grams(text):
  bi_token=[]
  bi_token_length=[]
  #print(len(text))
  for u in range(len(text)):
    sent_split=[w.lower() for w in text[u].split(" ")]
    sent_split1=[w for w in sent_split if w not in stop_words and w not in punctuation and not w.isdigit()]
    #bigrams_list=list(nltk.bigrams(sent_split1))
    bigrams_list = list(ngrams(sent_split, 2)) 
    #print(bigrams_list)
    #bigrams_list = ngrams(sent_split1,2)
    #bigrams_list = [bigram for bigram in nltk.bigrams(sent_split1)]
    #print(bigrams_list)
    bi_token.append(bigrams_list)
    bi_token_length.append(len(bi_token[u]))
  #print(bi_token_length)
  bi_tokens = [(int(m) / max(bi_token_length))*100 for m in bi_token_length]
  return bi_token_length
      

df["bigrams"] = df["Lemmatize"].apply(lambda text: bi_grams(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]"
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ..."


trigrams

In [12]:
stop_words = set(stopwords.words('english'))

def tri_grams(text):
  tri_token=[]
  tri_token_length=[]
  #print(len(text))
  for u in range(len(text)):
    sent_split2=[w.lower() for w in text[u].split(" ")]
    sent_split3=[w for w in sent_split2 if w not in stop_words and w not in punctuation and not w.isdigit()]
    
    try :
      trigrams_list = list(ngrams((sent_split3),3))
    except:
       print("Error is :",sent_split3)
    #print(trigrams_list)
    #trigrams_list = extract_ngrams(sent_split2, 3)
    #print(trigrams_list)
    #trigrams_list = ngrams(sent_split2,3)
    tri_token.append(trigrams_list)
    #print(len(tri_token[u]))
    l=len(tri_token[u])
    tri_token_length.append(l)
  tri_tokens = [(int(m) / max(tri_token_length))*100 for m in tri_token_length]
  return tri_token_length
      

df["trigrams"] = df["Lemmatize"].apply(lambda text: tri_grams(text))
df.head()

Error is : ['asked.']
Error is : ['brebner.']
Error is : ['up?']
Error is : ['bit.']
Error is : ['said.']
Error is : ['twice.']
Error is : ['bosvelt.']
Error is : ['possible."']
Error is : ['growing.']
Error is : ['ipod?']
Error is : ['gps?']
Error is : ['competition.']
Error is : ['why?']
Error is : ['shock.']
Error is : ['happen?']
Error is : ['friends.']
Error is : ['me."']
Error is : ['interested."']
Error is : ['can."']
Error is : ['worrying."']
Error is : ['asked.']
Error is : ['djs.']
Error is : ['conceived.']
Error is : ['lives."']
Error is : ['public.']
Error is : ['not?"']
Error is : ['said.']
Error is : ['said.']
Error is : ['added.']
Error is : ['saying?']
Error is : ['overall.']
Error is : ['that.']
Error is : ['time.']
Error is : ['work?']
Error is : ['polite.']
Error is : ['gps?']
Error is : ['competition.']
Error is : ['why?']
Error is : ['cole.']
Error is : ['delighted."']
Error is : ['said.']
Error is : ['"ditto."']
Error is : ['asked.']
Error is : ['congratulations!'

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]"
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]"
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ...","[16, 12, 14, 11, 13, 5, 4, 13, 5, 8, 33, 12, 1..."


Sentence position

In [13]:
def position(text):
  
  return [index for index, value in enumerate(text)]

def sent_count(text):
  return len(text)

def pos_vec(text):
  position = []
  position_rbm = []
  sent_pos1_rbm = 1
  sent_pos1 = 100
  sent_pos2 = 100
  sent_pos2_rbm = 1
  position.append(sent_pos1)
  position_rbm.append(sent_pos1_rbm)
  for x in range(1,text-1):
    s_p=((text-x)/text)*100
    position.append(s_p)
    s_p_rbm=(text-x)/text
    position_rbm.append(s_p_rbm)
  position.append(sent_pos2)
  position_rbm.append(sent_pos2_rbm)
  return position_rbm


df["sent_position"] = df["Lemmatize"].apply(lambda text: position(text))
df["num_of_sent"]= df["sent_position"].apply(lambda text: sent_count(text))
df["sent_posvec"]=df["num_of_sent"].apply(lambda text: pos_vec(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0...."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0...."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0...."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0...."
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ...","[16, 12, 14, 11, 13, 5, 4, 13, 5, 8, 33, 12, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26,"[1, 0.9615384615384616, 0.9230769230769231, 0...."


Converting sentences to vectors

In [14]:
def sent_to_vector(text):
  vocabulary = []
  for sents in text:
    vocabulary.extend(sents)
  vocabulary = list(set(vocabulary))
  vectors = []
  for sents in text:
    vector = []
    for tokenss in vocabulary:
      vector.append(sents.count(tokenss))
    vectors.append(vector)
  return vectors

df["sent_vector"]= df["Lemmatize"].apply(lambda text: sent_to_vector(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17..."
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ...","[16, 12, 14, 11, 13, 5, 4, 13, 5, 8, 33, 12, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26,"[1, 0.9615384615384616, 0.9230769230769231, 0....","[[7, 1, 0, 11, 0, 1, 0, 5, 2, 1, 2, 3, 0, 0, 6..."


tfisf

In [15]:
def tfisf(text):
  tfvec=[]
  isfvec=[]
  tfisfvec=[]
  tfisfvec_rbm=[]
  for i in range(len(text)):
    vocab_len = len(text[i])
    sentences_len = len(text)
    count = 0
    tfisf = 0
    for j in range(vocab_len):
      tf = text[i][j]
      if(tf>0):
        count += 1
        sent_freq = 0
        for k in range(sentences_len):
          if(text[k][j]>0): sent_freq += 1
        tfisf += (tf)*(1.0/sent_freq)
    if(count > 0):
      mean_tfisf = tfisf/count
    else:
      mean_tfisf = 0
    x,y,z= tf, (1.0/sent_freq), mean_tfisf
    tfvec.append(x)
    isfvec.append(y)
    tfisfvec.append(z*100)
    tfisfvec_rbm.append(z)

  return tfisfvec_rbm


df["tf_isf"]= df["sent_vector"].apply(lambda text: tfisf(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17..."
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ...","[16, 12, 14, 11, 13, 5, 4, 13, 5, 8, 33, 12, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26,"[1, 0.9615384615384616, 0.9230769230769231, 0....","[[7, 1, 0, 11, 0, 1, 0, 5, 2, 1, 2, 3, 0, 0, 6...","[0.2301825209350293, 0.17326017613202968, 0.17..."


max tfisf

In [16]:
def max_tfisf(text):
  maxtf_isf=max(text)
  centroid=[]
  centroid.append(maxtf_isf)
  return centroid

df["maxm_tfisf"]= df["tf_isf"].apply(lambda text: max_tfisf(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244]
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235]
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148]
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17...",[1.1294965786901272]
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ...","[16, 12, 14, 11, 13, 5, 4, 13, 5, 8, 33, 12, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26,"[1, 0.9615384615384616, 0.9230769230769231, 0....","[[7, 1, 0, 11, 0, 1, 0, 5, 2, 1, 2, 3, 0, 0, 6...","[0.2301825209350293, 0.17326017613202968, 0.17...",[0.43964910391540823]


centroid

In [17]:
def centroid(text):
  centroid=(max(text))
  return centroid

df["centrd"]= df["sent_vector"].apply(lambda text: centroid(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17...",[1.1294965786901272],"[15, 5, 5, 18, 2, 1, 18, 2, 2, 6, 5, 22, 13, 1..."
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ...","[16, 12, 14, 11, 13, 5, 4, 13, 5, 8, 33, 12, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26,"[1, 0.9615384615384616, 0.9230769230769231, 0....","[[7, 1, 0, 11, 0, 1, 0, 5, 2, 1, 2, 3, 0, 0, 6...","[0.2301825209350293, 0.17326017613202968, 0.17...",[0.43964910391540823],"[20, 4, 0, 15, 0, 2, 0, 10, 0, 3, 7, 2, 0, 1, ..."


Cosine similarity

In [18]:
def centroid(text):
  centroid=(max(text))
  return centroid
def cos_sim(text):
  cent=centroid(text)
  cosine_similarity=[]
  cosine_similarity_rbm=[]
  for z in range(len(text)):
    cos_simi = ((dot(cent, text[z])/(norm(cent)*norm(text[z])))*100)
    cosine_similarity.append(cos_simi)
    cos_simi_rbm = (dot(cent, text[z])/(norm(cent)*norm(text[z])))
    cosine_similarity_rbm.append(cos_simi_rbm)
 #     print("Cosine Similarity Vector:",cosine_similarity_rbm)
#    print("\n")
  return cosine_similarity_rbm

df["cossim"]= df["sent_vector"].apply(lambda text: cos_sim(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0...","[0.9700847260045529, 0.9260900228400718, 0.943..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,...","[0.8862933617642866, 0.8559813218781243, 0.850..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19...","[0.9112794729949737, 0.9310881101749274, 0.908..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17...",[1.1294965786901272],"[15, 5, 5, 18, 2, 1, 18, 2, 2, 6, 5, 22, 13, 1...","[0.9094703220070359, 0.8596974212723024, 0.862..."
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ...","[16, 12, 14, 11, 13, 5, 4, 13, 5, 8, 33, 12, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26,"[1, 0.9615384615384616, 0.9230769230769231, 0....","[[7, 1, 0, 11, 0, 1, 0, 5, 2, 1, 2, 3, 0, 0, 6...","[0.2301825209350293, 0.17326017613202968, 0.17...",[0.43964910391540823],"[20, 4, 0, 15, 0, 2, 0, 10, 0, 3, 7, 2, 0, 1, ...","[0.9343813349716937, 0.9351951135567246, 0.940..."


Sentence Length

In [19]:
def sentencelen(text):
  sent_word=[]
  for u in range(len(text)):
    sent_split1=[w.lower() for w in text[u].split(" ")]
    sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
    a=(len(sent_split))
    sent_word.append(a)
  longest_sent=max(sent_word)
  sent_length=[]
  sent_length_rbm=[]
  for x in sent_word:
    sent_length.append((x/longest_sent)*100)
    sent_length_rbm.append(x/longest_sent)
        #print(sent_length)
  return sent_length_rbm

df["sent_length"]= df["Lemmatize"].apply(lambda text: sentencelen(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim,sent_length
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0...","[0.9700847260045529, 0.9260900228400718, 0.943...","[1.0, 0.35714285714285715, 0.6071428571428571,..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,...","[0.8862933617642866, 0.8559813218781243, 0.850...","[1.0, 0.6842105263157895, 0.5263157894736842, ..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19...","[0.9112794729949737, 0.9310881101749274, 0.908...","[0.95, 0.75, 0.45, 0.65, 0.5, 0.8, 0.4, 1.0, 0..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17...",[1.1294965786901272],"[15, 5, 5, 18, 2, 1, 18, 2, 2, 6, 5, 22, 13, 1...","[0.9094703220070359, 0.8596974212723024, 0.862...","[0.5294117647058824, 0.17647058823529413, 0.14..."
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ...","[16, 12, 14, 11, 13, 5, 4, 13, 5, 8, 33, 12, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",26,"[1, 0.9615384615384616, 0.9230769230769231, 0....","[[7, 1, 0, 11, 0, 1, 0, 5, 2, 1, 2, 3, 0, 0, 6...","[0.2301825209350293, 0.173

Numeric token

In [20]:

def sentword(text):
  sent_word=[]
  for u in range(len(text)):
      sent_split1=[w.lower() for w in text[u].split(" ")]
      sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
      a=(len(sent_split))
      sent_word.append(a)
  #print(sent_word)
  return sent_word
def numeric_token(text):
  
  num_word=[]
  numeric_token=[]
  numeric_token_rbm=[]
  sd=sentword(text)
  #print(sd)
  for u in range(len(text)):
    #sd=sentword(text[u])
    
    sent_split4=text[u].split(" ")
    e=re.findall("\d+",text[u])
    noofwords=(len(e))
    num_word.append(noofwords)
    try:
      numeric_token.append((num_word[u]/sd[u])*100)
    except:
      print("Error is:", sd[u])
    try:
      numeric_token_rbm.append(num_word[u]/sd[u])
    except:
      print("Error is:", sd[u]) 
  return numeric_token_rbm

df["num_token"]= df["Lemmatize"].apply(lambda text: numeric_token(text))
df.head()

Error is: 0
Error is: 0


,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim,sent_length,num_token
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0...","[0.9700847260045529, 0.9260900228400718, 0.943...","[1.0, 0.35714285714285715, 0.6071428571428571,...","[0.03571428571428571, 0.0, 0.05882352941176470..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,...","[0.8862933617642866, 0.8559813218781243, 0.850...","[1.0, 0.6842105263157895, 0.5263157894736842, ...","[0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.0,..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19...","[0.9112794729949737, 0.9310881101749274, 0.908...","[0.95, 0.75, 0.45, 0.65, 0.5, 0.8, 0.4, 1.0, 0...","[0.10526315789473684, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17...",[1.1294965786901272],"[15, 5, 5, 18, 2, 1, 18, 2, 2, 6, 5, 22, 13, 1...","[0.9094703220070359, 0.8596974212723024, 0.862...","[0.5294117647058824, 0.17647058823529413, 0.14...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,[federer joins all-time greats\n \n the last y...,"[18, 13, 16, 12, 14, 7, 6, 14, 6, 10, 36, 13, ...","[16, 12, 14, 11, 13

Thematic Words

In [21]:
def thm_word_ftr(text):
      r = Rake() # Uses stopwords for english from NLTK, and all puntuation characters.    *********optimise with our own*********
      keywords=[]
      for s in text:
          r.extract_keywords_from_text(s)
          key=list(r.get_ranked_phrases())
          keywords.append(key)
      #print(keywords)
      l_keywords=[]
      for s in keywords:
          leng=len(s)
          l_keywords.append(leng)
      #print(l_keywords)

      total_keywords=sum(l_keywords)
      #print(total_keywords)

      thematic_number= []
      thematic_number_rbm= []
      for x in l_keywords:
          thematic_number.append((x/total_keywords)*100)
          thematic_number_rbm.append(x/total_keywords)
      #print("Thematic word feature", thematic_number_rbm)
      #print("\n")
      return thematic_number_rbm


df["thematic_word"]= df["Lemmatize"].apply(lambda text:thm_word_ftr(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim,sent_length,num_token,thematic_word
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0...","[0.9700847260045529, 0.9260900228400718, 0.943...","[1.0, 0.35714285714285715, 0.6071428571428571,...","[0.03571428571428571, 0.0, 0.05882352941176470...","[0.0847457627118644, 0.01694915254237288, 0.05..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,...","[0.8862933617642866, 0.8559813218781243, 0.850...","[1.0, 0.6842105263157895, 0.5263157894736842, ...","[0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.0,...","[0.08163265306122448, 0.10204081632653061, 0.0..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19...","[0.9112794729949737, 0.9310881101749274, 0.908...","[0.95, 0.75, 0.45, 0.65, 0.5, 0.8, 0.4, 1.0, 0...","[0.10526315789473684, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.043478260869565216, 0.06521739130434782, 0...."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17...",[1.1294965786901272],"[15, 5, 5, 18, 2, 1, 18, 2, 2, 6, 5, 22, 13, 1...","[0.9094703220070359, 0.8596974212723024, 0.862...","[0.5294117647058824, 0.17647058823529413, 0.14...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.09375, 0.09375, 0.03125, 0.03125, 0.03125, ..."
4,4,Federer joins all-time greats\n \n The last ye...,Andy Roddick was the only player to put up any...,3,sport,federer joins all-time greats\n \n the last ye...,

Proper noun

In [22]:
def sentword(text):
  sent_word=[]
  for u in range(len(text)):
      sent_split1=[w.lower() for w in text[u].split(" ")]
      sent_split=[w for w in sent_split1 if w not in stop_words and w not in punctuation and not w.isdigit()]
      a=(len(sent_split))
      sent_word.append(a)
  #print(sent_word)
  return sent_word
def pronoun(text):
  pncounts = []
  pncounts_rbm = []
  pnounscore_rbm = []
  sd=sentword(text)
  for sent in text:
    tagged=nltk.pos_tag(nltk.word_tokenize(str(sent)))
    counts = Counter(tag for word,tag in tagged if tag.startswith('NNP') or tag.startswith('NNPS'))
    f=sum(counts.values())
    pncounts.append(f)
    pncounts_rbm.append(f)
  try:
    pnounscore=[(int(o) / int(p))*100 for o,p in zip(pncounts, sd)]
    pnounscore_rbm=[int(o) / int(p) for o,p in zip(pncounts_rbm, sd)]
  except:
    print("Error:",sd)
      #print(pncounts)
      #print("Pronoun feature vector",pnounscore_rbm)
      #print("\n")
  return pnounscore_rbm

df["proper_noun"]= df["Lemmatize"].apply(lambda text:pronoun(text))
df.head()



Error: [22, 15, 13, 7, 15, 5, 11, 12, 11, 19, 23, 16, 12, 19, 10, 11, 17, 11, 12, 8, 13, 9, 3, 17, 0]


,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim,sent_length,num_token,thematic_word,proper_noun
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0...","[0.9700847260045529, 0.9260900228400718, 0.943...","[1.0, 0.35714285714285715, 0.6071428571428571,...","[0.03571428571428571, 0.0, 0.05882352941176470...","[0.0847457627118644, 0.01694915254237288, 0.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,...","[0.8862933617642866, 0.8559813218781243, 0.850...","[1.0, 0.6842105263157895, 0.5263157894736842, ...","[0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.0,...","[0.08163265306122448, 0.10204081632653061, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19...","[0.9112794729949737, 0.9310881101749274, 0.908...","[0.95, 0.75, 0.45, 0.65, 0.5, 0.8, 0.4, 1.0, 0...","[0.10526315789473684, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.043478260869565216, 0.06521739130434782, 0....","[0.0, 0.0, 0.0, 0.07692307692307693, 0.0, 0.0,..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17...",[1.1294965786901272],"[15, 5, 5, 18, 2, 1, 18, 2, 2, 6, 5, 22, 13, 1...","[0.9094703220070359, 0.8596974212723024, 0.862...","[0.5294117647058824, 0.17647058823529413, 0.14...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.09375, 0.09375, 0.03125, 0.03125, 0.03125,

***FEATURE MATRIX FORMATION***

In [24]:
def ftrmat(text):

    featureMatrix = []
    bi_token_length=bi_grams(text)
    tri_token_length=tri_grams(text)
    thematic_number_rbm=thm_word_ftr(text)
    sent_length_rbm=sentencelen(text)
    numeric_token_rbm=numeric_token(text)
    pnounscore_rbm=pronoun(text)
    sentvec=sent_to_vector(text)
    cosine_similarity_rbm=cos_sim(sentvec)
    tfisfvec_rbm=tfisf(sentvec)
    sentpos=position(text)
    num_sent=sent_count(sentpos)
    position_rbm=pos_vec(num_sent)




    featureMatrix.append(position_rbm)
    featureMatrix.append(bi_token_length)
    featureMatrix.append(tri_token_length)
    featureMatrix.append(tfisfvec_rbm)
    featureMatrix.append(cosine_similarity_rbm)
    featureMatrix.append(thematic_number_rbm)
    featureMatrix.append(sent_length_rbm)
    featureMatrix.append(numeric_token_rbm)
    featureMatrix.append(pnounscore_rbm)

    try:
      featureMat = np.zeros((len(text),9))
      for i in range(9) :
        for j in range(len(text)):
          featureMat[j][i] = featureMatrix[i][j]
    except:
      print("index out error")
    return featureMatrix

df["ft_mat"]= df["Lemmatize"].apply(lambda text:ftrmat(text))
df.head()

Error is : ['asked.']
Error is : ['brebner.']
Error is : ['up?']
Error is : ['bit.']
Error is : ['said.']
Error is : ['twice.']
Error is : ['bosvelt.']
Error is : ['possible."']
Error is : ['growing.']
Error is : ['ipod?']
Error is : ['gps?']
Error is : ['competition.']
Error is : ['why?']
Error is : ['shock.']
Error is : ['happen?']
Error is : ['friends.']
Error is : ['me."']
Error is : ['interested."']
Error is : ['can."']
Error is : ['worrying."']
Error is : ['asked.']
Error is : ['djs.']
Error is : ['conceived.']
Error is : ['lives."']
Error is : ['public.']
Error is : ['not?"']
Error is : ['said.']
Error is : ['said.']
Error is : ['added.']
Error is : ['saying?']
Error is : ['overall.']
Error is : ['that.']
Error is : ['time.']
Error is : ['work?']
Error is : ['polite.']
Error is : ['gps?']
Error is : ['competition.']
Error is : ['why?']
Error is : ['cole.']
Error is : ['delighted."']
Error is : ['said.']
Error is : ['"ditto."']
Error is : ['asked.']
Error is : ['congratulations!'

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim,sent_length,num_token,thematic_word,proper_noun,ft_mat
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0...","[0.9700847260045529, 0.9260900228400718, 0.943...","[1.0, 0.35714285714285715, 0.6071428571428571,...","[0.03571428571428571, 0.0, 0.05882352941176470...","[0.0847457627118644, 0.01694915254237288, 0.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9565217391304348, 0.9130434782608695, 0..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,...","[0.8862933617642866, 0.8559813218781243, 0.850...","[1.0, 0.6842105263157895, 0.5263157894736842, ...","[0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.0,...","[0.08163265306122448, 0.10204081632653061, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9285714285714286, 0.8571428571428571, 0..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19...","[0.9112794729949737, 0.9310881101749274, 0.908...","[0.95, 0.75, 0.45, 0.65, 0.5, 0.8, 0.4, 1.0, 0...","[0.10526315789473684, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.043478260869565216, 0.06521739130434782, 0....","[0.0, 0.0, 0.0, 0.07692307692307693, 0.0, 0.0,...","[[1, 0.9333333333333333, 0.8666666666666667, 0..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17...",[1.1294965786901272],"[15, 5, 5, 18, 2, 1, 18, 2, 2, 6, 5, 22, 13, 1...","[0.9094703220070359, 0.859697421272302

In [25]:
df["ft_mat"]

0      [[1, 0.9565217391304348, 0.9130434782608695, 0...
1      [[1, 0.9285714285714286, 0.8571428571428571, 0...
2      [[1, 0.9333333333333333, 0.8666666666666667, 0...
3      [[1, 0.9166666666666666, 0.8333333333333334, 0...
4      [[1, 0.9615384615384616, 0.9230769230769231, 0...
                             ...                        
440    [[1, 0.9615384615384616, 0.9230769230769231, 0...
441    [[1, 0.9444444444444444, 0.8888888888888888, 0...
442    [[1, 0.9090909090909091, 0.8181818181818182, 0...
443    [[1, 0.75, 0.5, 1], [15, 13, 10, 10], [14, 11,...
444    [[1, 0.9333333333333333, 0.8666666666666667, 0...
Name: ft_mat, Length: 2225, dtype: object

In [26]:
df['ft_mat'][1]

1    [[1, 0.9285714285714286, 0.8571428571428571, 0...
1    [[1, 0.9545454545454546, 0.9090909090909091, 0...
Name: ft_mat, dtype: object

normalised feature matrix

In [27]:
def sent_count(text):
  return len(text)

df["lemma_count"]= df["Lemmatize"].apply(lambda text: sent_count(text))
df["token_count"]= df["sen_token"].apply(lambda text: sent_count(text))
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim,sent_length,num_token,thematic_word,proper_noun,ft_mat,lemma_count,token_count
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0...","[0.9700847260045529, 0.9260900228400718, 0.943...","[1.0, 0.35714285714285715, 0.6071428571428571,...","[0.03571428571428571, 0.0, 0.05882352941176470...","[0.0847457627118644, 0.01694915254237288, 0.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9565217391304348, 0.9130434782608695, 0...",23,23
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,...","[0.8862933617642866, 0.8559813218781243, 0.850...","[1.0, 0.6842105263157895, 0.5263157894736842, ...","[0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.0,...","[0.08163265306122448, 0.10204081632653061, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9285714285714286, 0.8571428571428571, 0...",14,14
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19...","[0.9112794729949737, 0.9310881101749274, 0.908...","[0.95, 0.75, 0.45, 0.65, 0.5, 0.8, 0.4, 1.0, 0...","[0.10526315789473684, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.043478260869565216, 0.06521739130434782, 0....","[0.0, 0.0, 0.0, 0.07692307692307693, 0.0, 0.0,...","[[1, 0.9333333333333333, 0.8666666666666667, 0...",15,15
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 1, 0, 7, 0, 1, 9, 1, 1, 1, 2, 5, 3, 0, 17...","[0.5987440542996099, 0.25370851370851366, 0.17...",[1.1294965786901272],"[15, 5, 5, 18, 2, 1, 18, 2, 2, 6, 5, 22, 13, 1..

Normalised feature matrix

In [28]:
def normftrmat(text):
  len=sent_count(text)
  featureMat = np.zeros((len+1,9))
  #featureMat=[]
  
 # for i in range(9) :
  #  for j in range(len+1):
   #   featureMat[j][i] = text[i][j]
  featureMat=np.transpose(text)
  #temp = rbm.test_rbm(dataset = featureMat, learning_rate=0.01, training_epochs=15, batch_size=5,n_chains=9, n_hidden=9)
  return featureMat 
  
df["norm_ft_mat"]= df["ft_mat"].apply(lambda text:normftrmat(text))
df.head()

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim,sent_length,num_token,thematic_word,proper_noun,ft_mat,lemma_count,token_count,norm_ft_mat
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0...","[0.9700847260045529, 0.9260900228400718, 0.943...","[1.0, 0.35714285714285715, 0.6071428571428571,...","[0.03571428571428571, 0.0, 0.05882352941176470...","[0.0847457627118644, 0.01694915254237288, 0.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9565217391304348, 0.9130434782608695, 0...",23,23,"[[1.0, 29.0, 26.0, 0.5564895673019244, 0.97008..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,...","[0.8862933617642866, 0.8559813218781243, 0.850...","[1.0, 0.6842105263157895, 0.5263157894736842, ...","[0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.0,...","[0.08163265306122448, 0.10204081632653061, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9285714285714286, 0.8571428571428571, 0...",14,14,"[[1.0, 18.0, 17.0, 0.5197140466371235, 0.88629..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19...","[0.9112794729949737, 0.9310881101749274, 0.908...","[0.95, 0.75, 0.45, 0.65, 0.5, 0.8, 0.4, 1.0, 0...","[0.10526315789473684, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.043478260869565216, 0.06521739130434782, 0....","[0.0, 0.0, 0.0, 0.07692307692307693, 0.0, 0.0,...","[[1, 0.9333333333333333, 0.8666666666666667, 0...",15,15,"[[1.0, 18.0, 17.0, 0.5094978434264148, 0.91127..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15, 13, 10, 36, 6, 8]","[16, 4, 3, 7, 4, 7, 14, 11, 8, 32, 5, 6]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]",12,"[1, 0.9166666666666666, 0.8333333333333334, 0....","[[7, 

In [29]:
def ftnormed(text):
  featureMat_normed = text

  
  return featureMat_normed

df["ftnormed"]= df["norm_ft_mat"].apply(lambda text:ftnormed(text))
df.head()


,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim,sent_length,num_token,thematic_word,proper_noun,ft_mat,lemma_count,token_count,norm_ft_mat,ftnormed
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[13, 6, 14, 0, 2, 17, 0, 2, 7, 4, 1, 8, 20, 0...","[0.5564895673019244, 0.16396051536471756, 0.33...",[0.5564895673019244],"[14, 6, 13, 0, 4, 17, 0, 2, 11, 4, 0, 9, 20, 0...","[0.9700847260045529, 0.9260900228400718, 0.943...","[1.0, 0.35714285714285715, 0.6071428571428571,...","[0.03571428571428571, 0.0, 0.05882352941176470...","[0.0847457627118644, 0.01694915254237288, 0.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9565217391304348, 0.9130434782608695, 0...",23,23,"[[1.0, 29.0, 26.0, 0.5564895673019244, 0.97008...","[[1.0, 29.0, 26.0, 0.5564895673019244, 0.97008..."
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[4, 4, 8, 0, 1, 14, 2, 1, 5, 1, 0, 7, 13, 18,...","[0.5197140466371235, 0.31443278943278946, 0.24...",[0.5197140466371235],"[7, 4, 4, 0, 2, 5, 0, 1, 0, 2, 0, 6, 5, 13, 0,...","[0.8862933617642866, 0.8559813218781243, 0.850...","[1.0, 0.6842105263157895, 0.5263157894736842, ...","[0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.0,...","[0.08163265306122448, 0.10204081632653061, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9285714285714286, 0.8571428571428571, 0...",14,14,"[[1.0, 18.0, 17.0, 0.5197140466371235, 0.88629...","[[1.0, 18.0, 17.0, 0.5197140466371235, 0.88629..."
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[4, 3, 8, 0, 1, 0, 6, 0, 0, 4, 1, 2, 13, 10, ...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 5, 6, 0, 2, 0, 7, 1, 3, 3, 3, 0, 10, 7, 19...","[0.9112794729949737, 0.9310881101749274, 0.908...","[0.95, 0.75, 0.45, 0.65, 0.5, 0.8, 0.4, 1.0, 0...","[0.10526315789473684, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.043478260869565216, 0.06521739130434782, 0....","[0.0, 0.0, 0.0, 0.07692307692307693, 0.0, 0.0,...","[[1, 0.9333333333333333, 0.8666666666666667, 0...",15,15,"[[1.0, 18.0, 17.0, 0.5094978434264148, 0.91127...","[[1.0, 18.0, 17.0, 0.5094978434264148, 0.91127..."
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,[zambia confident cautious\n \n zambia's techn...,"[17, 5, 4, 8, 5, 8, 15,

In [30]:
import theano.tensor as T

Import python file from drive

In [1]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
!sudo pip install nltk==3.4

     |████████████████████████████████| 1.4MB 5.1MB/s 
  Created wheel for nltk: filename=nltk-3.4-cp37-none-any.whl size=1436382 sha256=e078f78afa22dc8ae0981354343a70e3d4004873a5005ab98ec2b5a7f739e119
  Stored in directory: /root/.cache/pip/wheels/4b/c8/24/b2343664bcceb7147efeb21c0b23703a05b23fcfeaceaa2a1e
Successfully built nltk
  Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


Logistic_sgd module

In [2]:
logistic_sgd=drive.CreateFile({'id':'1QfMYnBYaIBMUQfwzkUW-h_8ESNQHZkS0'})
logistic_sgd.GetContentFile("logistic_sgd.py")


In [3]:
shared_randomstreams=drive.CreateFile({'id':'1XQDMQ2lnc5ZEfJR0aivJTyNCHBcK1POs'})
shared_randomstreams.GetContentFile("shared_randomstreams.py")

In [4]:
rbm=drive.CreateFile({'id':'1jQaQl7MZ93urOgSyIpuZ8PAlTuUcoizK'})
rbm.GetContentFile("rbm.py")

In [35]:
!pip3 install theano

     |████████████████████████████████| 2.8MB 4.1MB/s 
  Created wheel for theano: filename=Theano-1.0.5-cp37-none-any.whl size=2668094 sha256=03383e94eef5f9a8eb20c5300ec0c342b57e725d8347b972972e4b852b3f2c9f
  Stored in directory: /root/.cache/pip/wheels/89/40/74/3a0b7d937890c66c4373120117ebf4ba99f4889b4a0a6a810c
Successfully built theano


In [5]:
from theano.tensor.shared_randomstreams import RandomStreams

In [6]:
import rbm

In [7]:
def emptyarray(text):
  emptyarray= np.empty((len(text),1,3),dtype=object)
  for s in range(len(text)):
      emptyarray[s][0][0] = text[s]
      emptyarray[s][0][1] = s
  return emptyarray

df["empty_array"]= df["Lemmatize"].apply(lambda text: emptyarray(text))


NameError: ignored

In [ ]:
df.head()

,Unnamed: 0,News,Summary,Category_id,Type,rem_stopwords,sen_token,Porter_stemmed,Lemmatize,bigrams,trigrams,sent_position,num_of_sent,sent_posvec,sent_vector,tf_isf,maxm_tfisf,centrd,cossim,sent_length,num_token,thematic_word,proper_noun,ft_mat,lemma_count,token_count,norm_ft_mat,ftnormed,empty_array
0,0,Online games play with politics\n \n After bub...,"Dr Frasca said: ""Political campaigns will cont...",4,tech,online games play politics\n \n after bubbling...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,[online games play politics\n \n after bubblin...,"[29, 9, 17, 19, 9, 12, 27, 18, 8, 7, 13, 18, 1...","[26, 8, 15, 16, 8, 10, 24, 16, 7, 5, 11, 17, 1...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",23,"[1, 0.9565217391304348, 0.9130434782608695, 0....","[[2, 1, 0, 2, 11, 6, 2, 2, 10, 7, 4, 1, 15, 12...","[0.5564895673019243, 0.1639605153647176, 0.337...",[0.5564895673019243],"[6, 0, 0, 2, 7, 4, 0, 2, 6, 4, 0, 0, 7, 7, 1, ...","[0.9539452110617865, 0.9412521771661028, 0.926...","[1.0, 0.35714285714285715, 0.6071428571428571,...","[0.03571428571428571, 0.0, 0.05882352941176470...","[0.0847457627118644, 0.01694915254237288, 0.05...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9565217391304348, 0.9130434782608695, 0...",23,23,"[[1.0, 29.0, 26.0, 0.5564895673019243, 0.95394...","[[1.0, 29.0, 26.0, 0.5564895673019243, 0.95394...",[[[online games play politics\n \n after bubbl...
1,1,Hague 'given up' his PM ambition\n \n Former C...,Mr Hague became an MP at 27 and Leader of the ...,2,politics,hague 'given up' pm ambition\n \n former conse...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,[hague 'given up' pm ambition\n \n former cons...,"[18, 12, 9, 13, 14, 12, 8, 1, 13, 16, 19, 8, 7...","[17, 11, 8, 11, 13, 9, 7, 0, 11, 15, 16, 6, 6, 4]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]",14,"[1, 0.9285714285714286, 0.8571428571428571, 0....","[[3, 0, 1, 4, 7, 3, 0, 6, 5, 0, 2, 0, 12, 5, 1...","[0.5197140466371235, 0.3144327894327894, 0.244...",[0.5197140466371235],"[5, 0, 3, 4, 2, 4, 0, 6, 2, 0, 0, 0, 7, 8, 1, ...","[0.8869837454064581, 0.8636063715376114, 0.923...","[1.0, 0.6842105263157895, 0.5263157894736842, ...","[0.0, 0.07692307692307693, 0.0, 0.0, 0.0, 0.0,...","[0.08163265306122448, 0.10204081632653061, 0.0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[[1, 0.9285714285714286, 0.8571428571428571, 0...",14,14,"[[1.0, 18.0, 17.0, 0.5197140466371235, 0.88698...","[[1.0, 18.0, 17.0, 0.5197140466371235, 0.88698...",[[[hague 'given up' pm ambition\n \n former co...
2,2,M&S cuts prices by average of 24%\n \n Marks &...,"""We are bringing in ranges at new price points...",0,business,m&s cuts prices average 24%\n \n marks spencer...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,[m&s cuts prices average 24%\n \n marks spence...,"[18, 14, 8, 12, 9, 15, 7, 19, 18, 18, 14, 15, ...","[17, 13, 7, 11, 8, 14, 6, 18, 17, 17, 12, 13, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...",15,"[1, 0.9333333333333333, 0.8666666666666667, 0....","[[2, 2, 0, 0, 1, 3, 1, 9, 4, 2, 2, 5, 8, 1, 3,...","[0.5094978434264148, 0.3527169497169497, 0.174...",[0.5094978434264148],"[6, 0, 0, 1, 6, 4, 3, 7, 4, 0, 0, 12, 7, 1, 2,...","[0.9155950362946681, 0.9316986368122273, 0.852...","[0.95, 0.75, 0.45, 0.65, 0.5, 0.8, 0.4, 1.0, 0...","[0.10526315789473684, 0.0, 0.0, 0.0, 0.0, 0.0,...","[0.043478260869565216, 0.06521739130434782, 0....","[0.0, 0.0, 0.0, 0.07692307692307693, 0.0, 0.0,...","[[1, 0.9333333333333333, 0.8666666666666667, 0...",15,15,"[[1.0, 18.0, 17.0, 0.5094978434264148, 0.91559...","[[1.0, 18.0, 17.0, 0.5094978434264148, 0.91559...",[[[m&s cuts prices average 24%\n \n marks spen...
3,3,Zambia confident and cautious\n \n Zambia's te...,That victory for Angola also marked a first de...,3,sport,zambia confident cautious\n \n zambia's techni...,[zambia conf

In [ ]:
def enh(text):
  temp = rbm.test_rbm(dataset, learning_rate=0.01, training_epochs=15, batch_size=5,n_chains=9, n_hidden=9)
  enhanced_feature_sum = []
  for i in range(len(np.sum(temp,axis=1))) :
    enhanced_feature_sum.append([np.sum(temp,axis=1)[i],i])
    emptyarray[i][0][2]=np.sum(temp,axis=1)[i]
  enhanced_feature_sum.sort(key=lambda x: x[0])
  return enhanced_feature_sum

df["enh"]= df["norm_ft_mat"].apply(lambda text: enh(text))



NameError: ignored

Rough work

In [ ]:
ftmatrixnormed=df['norm_ft_mat'][0]
ftmatrixnormed

0    [[1.0, 29.0, 26.0, 0.5564895673019243, 0.95394...
0    [[1.0, 19.0, 17.0, 0.32923304902572637, 0.9408...
Name: norm_ft_mat, dtype: object

In [ ]:
temp = rbm.test_rbm(dataset = ftmatrixnormed, learning_rate=0.01, training_epochs=15, batch_size=5,n_chains=9, n_hidden=9)

TypeError: ignored

In [ ]:
def len_ext(text):
  length_to_be_extracted = len(enhanced_feature_sum)/2
  return length_to_be_extracted

